프로퍼티
프로퍼티도 클래스이다.
프로퍼티 클래스 사용
`=` 를 사용하면 `setter` 함수가 호출된다. = 연산자 오버로딩과 유사하다.



In [2]:
class Natural:

    def __init__(self,n):
        # 음수 차단
        if n < 0: 
            self.__n = 0
        else:
            self.__n = n

    def getn(self):
        return self.__n
    
    def setn(self,n):
        if n < 1:
            self.__n = 1
        else:
            self.__n = n
    
    p = property(getn, setn) #클래스 변수

n1 = Natural(1)
n2 = Natural(2)
n3 = Natural(3)


print(n1.getn())
print(n1.p) # .p가 단독으로 사용이되면 getter 함수가 호출된다.

n1.p = n2.p + n3.p # `.p`가 `=` 과 함께 사용되면 setter 함수가 호출된다.

print(n1.p)


#print(n2.getn())

1
1
5


프로퍼티 데코레이터를 사용한다.

In [4]:
class Citzen:
    def __init__(self,age_value):
        self.__age = age_value

    @property # 프로퍼티 함수에 age 함수를 전달
    def age(self):
        print("나이를 리턴합니다.")
        return self.__age
    
    @age.setter # age가 프로퍼티 객체가 되어서 안에있는 setter 함수를 호출
    def age(self, age_value):
        print("나이를 설정합니다.")
        self.__age = age_value
    
a = Citzen(20)
print(a.age) # .age는 변수가 아니라 프로퍼티 함수다.
a.age = 40
print(a.age)

나이를 리턴합니다.
20
나이를 설정합니다.
나이를 리턴합니다.
40


In [5]:
# 프로퍼티 클래스의 이해

# 디스크립터 클래스

class CharacterInfo:
    def __init__(self, power, speed):
        self.power = power
        self.speed = speed

    def __get__(self, obj, objtype):
        print('(GET)정보 조회됨')
        #<main.CharacterInfo object at 0x000001BCB2AD7D30> 
        # <main.Guardian object at 0x000001BCB2AD7BB0> 
        # <class 'main.Guardian'>
        print(self, obj, objtype)
        return ("공격력 : "+str(self.power) + " / 스피드 : " + self.speed)

    def __set__(self, obj, val):
        print('(UPDATE)정보 갱신 시작')
        self.power = val.power
        self.speed = val.speed

    def __delete__(self, obj):
        print("(DELETE)정보 삭제하기")
        self.power =""
        self.speed = ""

class Guardian:
    info = CharacterInfo(10, "50km/h")


g1 = Guardian()   # g1 이라는 수호천사 인스턴스 하나 생성
print(g1.info)   # 인스턴스 g1의 초기 정보 출력 


info_after_upgrade = CharacterInfo(15, "70km/h")   # 업그레이드 아이템 적용 후 캐릭터 정보
g1.info = info_after_upgrade   # 새 캐릭터 정보를 인스턴스 g1 에 설정
print(g1.info)   # 인스턴스 g1의 정보 출력


del g1.info   # 인스턴스 g1의 정보 삭제
print(g1.info)   # 인스턴스 g1의 정보 출력

(GET)정보 조회됨
<__main__.CharacterInfo object at 0x000001EC570AFBB0> <__main__.Guardian object at 0x000001EC570AD600> <class '__main__.Guardian'>
공격력 : 10 / 스피드 : 50km/h
(UPDATE)정보 갱신 시작
(GET)정보 조회됨
<__main__.CharacterInfo object at 0x000001EC570AFBB0> <__main__.Guardian object at 0x000001EC570AD600> <class '__main__.Guardian'>
공격력 : 15 / 스피드 : 70km/h
(DELETE)정보 삭제하기
(GET)정보 조회됨
<__main__.CharacterInfo object at 0x000001EC570AFBB0> <__main__.Guardian object at 0x000001EC570AD600> <class '__main__.Guardian'>
공격력 :  / 스피드 : 


In [ ]:
class Car:
    def __init__(self, initial_speed):
        self._speed = initial_speed

    @property #__get__
    def speed(self):
        """I'm the 'speed' property."""
        print('현재 속도 구하기')
        return self._speed

    @speed.setter #__set__
    def speed(self, value):
        print('현재 속도 설정하기')
        self._speed = value

    @speed.deleter #__delete__
    def speed(self):
        print('현재 속도 정보 삭제하기')
        del self._speed


car = Car(50)
print(car.speed)
car.speed = 100
print(car.speed)
del car.speed

In [ ]:
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.doc
        self.doc = doc

    def set_name(self, owner, name):
        self.name = name

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.doc)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.doc)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.doc)